### nbdc data

In [1]:
import requests
import pandas as pd
import numpy as np

### get all relevant buoys

In [26]:
text_data = requests.get('https://www.ndbc.noaa.gov/data/stations/station_table.txt').text
rows = text_data.strip().split('\n')

# extracting headers
headers = map(lambda x: x.strip(), rows[0].strip().split('|'))

# remaining data. (skipping with only `#`)
data = [row.strip().split('|') for row in rows[2:]]

df_buoys = pd.DataFrame(data, columns = headers)

### Get buoy data

In [92]:
def getBuoyData(buoyid): 
    # Split the text into rows using newline characters
    try: 
        rows = requests.get(f'https://www.ndbc.noaa.gov/data/realtime2/{buoyid}.spec')
        response.raise_for_status()
        rows = rows.text.strip().split('\n')
        print("success fetching this data")
    except: 
        raise("error fetching this data")
    # get headers, then the rest of the data
    headers = rows[0].strip().split()
    data = [row.split() for row in rows[1:]]

    df_buoy_data = pd.DataFrame(data, columns=headers)
    return df_buoy_data

In [93]:
getBuoyData(13001)

TypeError: exceptions must derive from BaseException

### Calculate that buoy's angle to Los Angeles

Swell direction is measured with north at 0 degrees, east at 90, etc. 

In [4]:
import math
import re

In [21]:
LA_lat = 33.8847
LA_long = -118.4109

def calculate_angle(coords):
    buoylat, buoylong = parse_coordinates(coords) if isinstance(coords, str) else coords
    LA_lat = 33.8847
    LA_long = -118.4109
    x = buoylong - LA_long
    y = buoylat - LA_lat
    
    if x >= 0:
        return 90 - math.atan(y / x) * 180 / math.pi 
    elif x < 0: 
        return 270 - math.atan(y / x) * 180 / math.pi
    
    #return math.atan(y / x) * 180 / math.pi
    
    
def parse_coordinates(input_string):
    pattern = r'(\d+\.\d+)\s*([NS])\s*(\d+\.\d+)\s*([EW])'
    match = re.search(pattern, input_string)
    if match:
        latitude = float(match.group(1))
        if match.group(2) == 'S':
            latitude = -latitude
        
        longitude = float(match.group(3))
        if match.group(4) == 'W':
            longitude = -longitude
        
        return latitude, longitude
    else:
        return None
    

In [22]:
df_buoys['angletoLA'] = df_buoys.apply(lambda row: calculate_angle(row.LOCATION), axis = 1)

### Get all valid buoys -- request all the data for those buoys!!

Buoys in between angle 160 and less than 360. longitude less than 0. 

In [81]:
def isValidBuoy(deg, long): 
    if deg >= 160 and deg <= 360 and long < 0: 
        return True
    return False


def builddata():
    df_main = pd.DataFrame(columns = df_buoy_data.columns.tolist())
    #for i in range(len(df_buoys)): 
    i = 1
    lat, long = parse_coordinates(df_buoys.LOCATION.iloc[i])
    angle = calculate_angle((lat, long))
    buoy_id = int(df_buoys.iloc[i, 0])
    print(buoy_id)
    temp = getBuoyData(buoy_id)
    if isValidBuoy(angle, long): 
        df_main.append(temp)
    return df_main

In [86]:
getBuoyData(13001)

ValueError: 6 columns passed, passed data had 10 columns

In [60]:
df_buoy_data.columns.tolist()

['#YY',
 'MM',
 'DD',
 'hh',
 'mm',
 'WVHT',
 'SwH',
 'SwP',
 'WWH',
 'WWP',
 'SwD',
 'WWD',
 'STEEPNESS',
 'APD',
 'MWD']

In [74]:
df_buoys.iloc[2, 0]

'13002'

In [72]:
df_buoys

,# STATION_ID,OWNER,TTYPE,HULL,NAME,PAYLOAD,LOCATION,TIMEZONE,FORECAST,NOTE
0,0y2w3,CG,Weather Station,,"Sturgeon Bay CG Station, WI",,"44.794 N 87.313 W (44&#176;47'39"" N 87&#176;18...",C,,
1,13001,PR,Atlas Buoy,PM-595,NE Extension,,"12.000 N 23.000 W (12&#176;0'0"" N 23&#176;0'0"" W)",,,
2,13002,PR,Atlas Buoy,,NE Extension,,"21.000 N 23.000 W (21&#176;0'0"" N 23&#176;0'0"" W)",,,
3,13008,PR,Atlas Buoy,PM-531,Reggae,,"15.000 N 38.000 W (15&#176;0'0"" N 38&#176;0'0"" W)",,,
4,13009,PR,Atlas Buoy,PM-533,Lambada,,"8.000 N 38.000 W (8&#176;0'0"" N 38&#176;0'0"" W)",,,
...,...,...,...,...,...,...,...,...,...,...
1823,ygnn6,EA,GLOS Weather Station,,"Niagara Coast Guard Station, NY",,"43.262 N 79.064 W (43&#176;15'42"" N 79&#176;3'...",E,,"For Great Lakes marine forecasts, select: <a h..."
1824,ykrv2,PT,Water Level Observation Network,,"8637611 - York River East Rear Range Light, VA",,"37.251 N 76.342 W (37&#176;15'5"" N 76&#176;20'...",E,FZUS51.KAKQ,
1825,yktv2,O,Water Level Observation Network,,"8637689 - Yorktown USCG Training Center, VA",,"37.227 N 76.479 W (37&#176;13'36"" N 76&#176;28...",E,FZUS51.KAKQ,
1826,yrsv2,NR,NERRS Weather Station,,"Taskinas Creek, Chesapeake Bay Reserve, VA",,"37.414 N 76.712 W (37&#176;24'51"" N 76&#176;42...",E,,Water Quality data for this Reserve are availa...
